In [1]:
import torch
import torch.nn as nn
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    device_map='auto'
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", 
                                         )


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
def get_llama2_response(prompt, max_new_tokens=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, temperature= 0.7)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [34]:
prompt = "Keep answer short. !!! Who kills Dumbledore at the end of 'Harry Potter and the Half-Blood Prince?'"
get_llama2_response(prompt, max_new_tokens=30)

"Keep answer short. !!! Who kills Dumbledore at the end of 'Harry Potter and the Half-Blood Prince?'\n\nAnswer: Severus Snape."

In [4]:
import json

def process_questions(input_file, output_file):
    with open(input_file, 'r') as file:
        data = json.load(file)
        questions = [item['question'] for item in data]
    
    # Dictionary to store responses
    responses = []
    # Generate responses for each question
    for question in questions:
        question = 'Keep answer short. [inst]' + question + '[inst]'
        response = get_llama2_response(question, max_new_tokens=30)
        splitter = '[inst]'
        response = response.replace('Keep answer short.', '').split(splitter)
        print(response)
        response = [item.strip() for item in response if item != ""]
        responses.append({"question": response[0].strip(), "response": response[1].strip()})
    
    # Save responses to a new JSON file
    with open(output_file, 'w') as file:
        json.dump(responses, file, indent=4)



In [5]:
input_json_file = "processed_qa_pairs.json"
output_json_file = "qa_pairs_llama2.json"
process_questions(input_json_file, output_json_file)

F:\NLP\nlp\.venv\Lib\site-packages\transformers\models\llama\modeling_llama.py:670: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


[' ', "'Who are Harry Potter's best friends?'", "  Harry Potter's best friends are Ron Weasley and Hermione Granger."]
[' ', "'What house is Harry Potter sorted into at Hogwarts?'", '  Harry Potter is sorted into Gryffindor House at Hogwarts School of Witchcraft and Wizardry.']
[' ', "'Who is the headmaster of Hogwarts when Harry first arrives?'", '  The headmaster of Hogwarts School of Witchcraft and Wizardry when Harry Potter first arrives is Professor Minerva McGon']
[' ', "'What position does Harry play on his Quidditch team?'", '  Harry plays seeker on his Quidditch team.']
[' ', "'What is the name of the dark wizard who is Harry's main antagonist?'", "  The dark wizard who is Harry's main antagonist is Lord Voldemort."]
[' ', "'What magical object did Harry inherit from his father, James?'", '  Harry inherited the Cloak of Invisibility from his father, James.']
[' ', "'Which creature gives birth to golden eggs, as seen in the Triwizard Tournament?'", '  The creature that gives bi

In [14]:
import json
import ast
import numpy as np

def parse_qa_from_file(input_file):
    qa_pairs = []
    with open(input_file, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for line in lines:
        line = np.array(ast.literal_eval(line))
        # if line.strip():  # Ensures that empty lines are skipped
        # Splitting the line at the newline and stripping any leading/trailing spaces
        if len(line) >= 3:
            question = line[1]
            answer = line[2]
            qa_pairs.append({"question": question, "answer": answer})

    return qa_pairs

def save_to_json(data, output_file):
    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(data, file, indent=4, ensure_ascii=False)

# File paths
input_filename = 'qa_pairs_llama2.txt'
output_filename = 'qa_pairs_llama2.json'

# Processing
qa_data = parse_qa_from_file(input_filename)
save_to_json(qa_data, output_filename)

print("QA data has been parsed and saved to JSON.")


QA data has been parsed and saved to JSON.
